# Linear Transformation 
We will start with the alignement of the same models for different seeds. 
- First we do not restrict the matrix.
- Second we restrict the matrix to be a rotation matrix.
- Thrid use affine transformation 
- Last but not least we are using different norms and regularization techniques to improve the results



Steps: 
- Load the same model but with different seed
- Sample different images and get latent representation 
- Create Datamatrix X and X'
- Solve the simple optimization problem (Using closed form solution as well cvxpy solver)


In [1]:
# Import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
import cvxpy as cp
import os
from optimizer import AffineFitting, LinearFitting
from utils.sampler import *
os.chdir('../')
print(f"Current working directory {os.getcwd()}")

Current working directory /mnt/c/Users/Hillary Hauger/Documents/Studium/SoSe2024/Case Study Non Linear Optimization/Code/latent-communication


In [2]:
# Configuration

config = {
    'path1': os.getcwd()+"/resnet/models/model_seed1.pth",
    'modelname1': 'resnet1',
    'path2': os.getcwd()+"/resnet/models/model_seed2.pth",
    'modelname2': 'resnet3',

}

## Load Model and Transformed Data for VAE
In this section we load the trained models, which we prepaired for our experimental setup

In [3]:
# Import Data 
from helper.DataLoaderMNIST import DataLoader_MNIST
# Transformations
transformations = [transforms.ToTensor(),
                    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
                    ]
# Load the data
data_loader = DataLoader_MNIST(128, transformations)

In [4]:
from resnet.model_def import ResNet
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the models
model = ResNet(pretrained=False).to(DEVICE)	
model.load_state_dict(torch.load(config['path1']))


# Initialize the model 2
model2 = ResNet(pretrained=False).to(DEVICE)# Load pretrained weights for model2
model2.load_state_dict(torch.load(config['path2']))

/home/hillary/miniconda3/envs/opti/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hillary/miniconda3/envs/opti/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

## Sampling 
We sample images from the train set and encode those for each model. In this section we implemented two sampler.
One just samples randomly and the second samples accordingly to the classes.

In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
z1,z2,images,labels = simple_sampler(100, model, model2, data_loader, DEVICE)

/home/hillary/miniconda3/envs/opti/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


## Optimization

### Linear Transformation

In [6]:
#Do Linear fitting
lamda = 0.01
linear_fitting = LinearFitting(z1, z2, lamda)

In [7]:
linear_fitting.solve_problem()

Solving the problem
Defining the problem
                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) May 26 02:44:44 PM: Your problem has 262144 variables, 0 constraints, and 0 parameters.
(CVXPY) May 26 02:44:44 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 26 02:44:44 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 26 02:44:44 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 26 02:44:44 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 26 02:44:44 PM:

: 

In [ ]:
linear_fitting.print_results()

### Affine Transformation

In [7]:
lamda=0.1
# Create an instance of the AffineFitting class
affine_fitting = AffineFitting(z1, z2, lamda)

In [8]:
# Solve the optimization problem
affine_fitting.solve_problem()

: 

In [7]:
lamda=0.1
# Create an instance of the AffineFitting class
affine_fitting = AffineFitting(z1, z2, lamda)

# Solve the optimization problem
affine_fitting.solve_problem()

# Print the results
affine_fitting.print_results()

: 